In [ ]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Para PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo

# Reglas
from apyori import apriori

pd.set_option('display.max_columns', None)
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

In [ ]:
# Global Variables
seed = random.seed(123)

In [ ]:
# Variable classification
train_data = pd.read_csv('./data/train.csv', encoding = "ISO-8859-1")
test_data = pd.read_csv('./data/test.csv', encoding = "ISO-8859-1")
variables = pd.read_csv('./data/variables.txt', encoding = "ISO-8859-1")
quant_vars = list(variables.loc[(variables['Clasification'] == 'Cuantitativa')]['Variable'].values)
quali_vars = list(variables.loc[(variables['Clasification'] == 'Cualitativa')]['Variable'].values)[1:]

# Analisis Exploratorio

### Analizando las variables numericas

In [ ]:
train_data[quant_vars].describe()

In [ ]:
for var in quant_vars:
    data = train_data[var].dropna(how='all', axis=0)
    
    # Gráfico
    sns.displot(data, kde=True)

    # Mostrando normalidad
    print('\033[1m' + var + '\033[0m' + ': Kurtosis:', stats.kurtosis(data), 'Skewness:', stats.skew(data), '\n')

### Analizando las variables categoricas

In [ ]:
for var in quali_vars:
  plt.figure(figsize=(20,5))
  train_data[var].value_counts().plot(kind='bar')
  plt.show()

### Correlacion

In [ ]:
k = 10 #number of variables for heatmap
corrmat = train_data.corr()
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(train_data[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

### Obteniendo la relacion entre las variables mas significativas

In [ ]:
sns.pairplot(train_data[cols],hue="SalePrice")
plt.show()
# quant_vars

# Preprocesamiento PCA

In [ ]:
copied_train_data = train_data.copy()
copied_train_data = copied_train_data.fillna(0)
numeric = copied_train_data[quant_vars].copy()
numeric

In [ ]:
chi_cuadrado, p_valor = calculate_bartlett_sphericity(numeric)
chi_cuadrado, p_valor

In [ ]:
kmo,kmo_modelo = calculate_kmo(numeric)
kmo_modelo

El índice es mayor a 0.5, por lo que la adecuación muestral para un análisis factorial es aceptable. Probablemente esto nos indique que voy a tener buenos resultados haciendo el PCA

In [ ]:
pca_pipe = make_pipeline(StandardScaler(), PCA()) #Se escalan los datos y luego se le aplica PCA
pca_pipe.fit(numeric)
#Se extrae el modelo del pipeline
modelo_pca = pca_pipe.named_steps['pca']

index = []
for i in range(len(numeric.columns)):
    index.append('PC' + str(i + 1))

pd.DataFrame(
    data = modelo_pca.components_,
    columns = numeric.columns,
    index = index
)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
componentes = modelo_pca.components_
plt.imshow(componentes.T)
plt.yticks(range(len(numeric.columns)), numeric.columns)
plt.xticks(range(len(numeric.columns)), np.arange(modelo_pca.n_components_)+1)
plt.grid(False)
fig.set_size_inches(19, 11, forward=True)
plt.colorbar()

In [ ]:
print('----------------------------------------------------')
print('Porcentaje de varianza explicada por cada componente')
print('----------------------------------------------------')
print(modelo_pca.explained_variance_ratio_)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))
ax.bar(
    x      = np.arange(modelo_pca.n_components_) + 1,
    height = modelo_pca.explained_variance_ratio_
)

for x, y in zip(np.arange(len(numeric.columns)) + 1, modelo_pca.explained_variance_ratio_):
    label = round(y, 2)
    ax.annotate(
        label,
        (x,y),
        textcoords="offset points",
        xytext=(0,10),
        ha='center'
    )

ax.set_xticks(np.arange(modelo_pca.n_components_) + 1)
ax.set_ylim(0, 1.1)
ax.set_title('Porcentaje de varianza explicada por cada componente')
ax.set_xlabel('Componente principal')
ax.set_ylabel('Por. varianza explicada')
fig.set_size_inches(15, 3, forward=True)

In [ ]:
# Porcentaje de varianza explicada acumulada
# ==============================================================================
prop_varianza_acum = modelo_pca.explained_variance_ratio_.cumsum()
print('------------------------------------------')
print('Porcentaje de varianza explicada acumulada')
print('------------------------------------------')
print(prop_varianza_acum)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))
ax.plot(
    np.arange(len(numeric.columns)) + 1,
    prop_varianza_acum,
    marker = 'o'
)

for x, y in zip(np.arange(len(numeric.columns)) + 1, prop_varianza_acum):
    label = round(y, 2)
    ax.annotate(
        label,
        (x,y),
        textcoords="offset points",
        xytext=(0,10),
        ha='center'
    )
    
ax.set_ylim(0, 1.1)
ax.set_xticks(np.arange(modelo_pca.n_components_) + 1)
ax.set_title('Porcentaje de varianza explicada acumulada')
ax.set_xlabel('Componente principal')
ax.set_ylabel('Por. varianza acumulada')
fig.set_size_inches(15, 5, forward=True)

# Preprocesamiento Reglas

In [ ]:
garage = 'GarageYrBlt'

copied_data = train_data.copy()
copied_data[garage] = copied_data[garage].fillna(0)
categories = copied_data[quali_vars].astype(object).copy()

categories[garage] = categories[garage].astype(np.int16)
categories[garage] = categories[garage].astype('string')
categories

In [ ]:
for var in quali_vars:
  print(categories[var].unique())

## Transformando los datos

In [ ]:
shape_row, shape_column = categories.shape
# Se van a convertir los datos a una lista de listas
records = []

for i in range(0, 32):
    records.append([str(categories.values[i,j]) for j in range(0, 5)]) 
records

In [ ]:
# El mínimo de cobertura o soporte es 20% y el mínimo de confianza es 70%
reglas_asociacion = apriori(records,min_support=0.2, min_confidence = 0.8)
reglas = list(reglas_asociacion)
len(reglas)

In [ ]:
print(reglas[0])

In [ ]:
list(reglas)